In [1]:
#%%
import sys
sys.path.append('../../models')
sys.path.append('../')


import datetime as dt
import math
##
import os
import random

import time

import lxml
import matplotlib.patches as patch
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.optimize as sc_opt

# Switching time optimization modules
import switching_times_1st as st1
import switching_times_1st_rk as st1_rk
import switching_times_2nd as st2
import switching_times_2nd_rk as st2_rk
import switching_times_3rd as st3
import switching_times_3rd_rk as st3_rk

import torch
from IPython.core.debugger import set_trace
from numpy import genfromtxt
from scipy.integrate import quad, solve_ivp
from sklearn.preprocessing import MinMaxScaler
from torch import nn, optim
from torch.autograd import Variable
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset

from ann.dense_nets import simpleNet
from data import SwitchData, SwitchData_rk
from model_loader import *
from models import firstordermodel, secondordermodel, thirdordermodel
from price_loader import *
from tools import (build_initial_ipopt_object, consMatrix, consumption, cost,
                   derive_regimes, discrete_ivp_solver, f_with_consumption,
                   f_with_cost, makeANNFeasible, mergeSwitchCont, plotSwitches,
                   predict_switch_ann_og, removeRedundantSwitches,
                   removeWrongOrder, sigmoid, simulate_MPC, smooth_dap,
                   smooth_regime, sol_ivp_wrapper, sol_ivp_wrapper_discrete,
                   stochasticSimulation, stochasticSimulation_IDLE)

sys.path.append('..')
%load_ext autoreload
%autoreload 2



os.system("../model_loader.py")
os.system("../price_loader.py")
os.system("../../models/models.py")


/Users/madsobdrup/opt/miniconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


32256

In [2]:
histories_02 = {}
histories_05 = {}
model_sys = 'm1'
n_s = 6
filename_02 = '../results/sim_history/og_history_(2018-01-01 12:00:00)_(100_days)' + '_(price_slope_' + str(0.2) +  ')_(regime_slope_' + str(0.2) +  ')_(seed_1235)_(n_s_' + str(n_s) + ')_(sys_model_' + model_sys + ')_(sys_true_m3).npy'
history = np.load(filename_02,allow_pickle=True).item()

n_s_all = history['SWITCHES_dap'].shape[1]

scaler = MinMaxScaler()
#switch_type = 'rk'

# Create training data set
batch_size = 4
train_range = range(0,80)
dset = SwitchData(history,train_range[0],train_range[-1]+1,transform=scaler.fit_transform )

In [3]:
# Load netowrk
model = simpleNet(n_s_all=12)
model.load_state_dict(torch.load('../ann/trained_networks/best_model_og_lossl2'))
model.eval()


k_baseline = 12400.
k_MELT = 250.
k_IDLE = 9.

model_sys = 1 #int(sys.argv[1])
n_days = 100 #int(sys.argv[2])
n_s = 6 #int(sys.argv[3])
true_sys = 3 #int(sys.argv[4])
price_slope = 0.2 #float(sys.argv[5])
regime_slope = 0.2 #float(sys.argv[6])

logfile = open("../results/current_run_rk_ANN_"+str(model_sys)+".txt","w")
logfile.write("CURRENT RUN""\n")
logfile.write("--------------""\n")
logfile.write("Model: " + str(model_sys) +  "\n")
logfile.write("Number of days: " + str(n_days) +  "\n")
logfile.write("n_s: " + str(n_s) +  "\n")
logfile.write("True: " + str(true_sys) +  "\n")
logfile.write("--------------""\n")
logfile.close()


# BUILD OPTIMIZATION FUNCTION
# Define parameters
#n_s = 6
max_melt = 16. * 60.
dt_opt = 0.1

# ------ Define models -------
# Expectation and optimization object model
# -------------
if model_sys == 1:
    print(1)
    system_model = m1; x0_model = x0_1; tank = st1_rk.plant(); tank_pars = m1.drift_pars; sys_mod = 'm1'
elif model_sys == 2:
    print(2)
    system_model = m2; x0_model = x0_2; tank = st2_rk.plant(); tank_pars = m2.drift_pars; sys_mod = 'm2'
elif model_sys == 3:
    print(3)
    system_model = m3; x0_model = x0_3; tank = st3_rk.plant(); tank_pars = m3.drift_pars; sys_mod = 'm3'
# -------------

# "True" model
# -------------
if true_sys == 1:
    print(1)
    system_true = m1; x0_true = x0_1; sys_true = 'm1'
elif true_sys == 2:
    print(2)
    system_true = m2; x0_true = x0_2; sys_true = 'm2'
elif true_sys == 3:
    print(3)
    system_true = m3; x0_true = x0_3; sys_true = 'm3'
elif true_sys == 4: # With diffusion 0
    print(4)
    system_true = m3_zero_diff; x0_true = x0_3; sys_true = 'm4'
# -------------


#%%

#%%
# Build object
t0 = 0.
tf_sim = 60 * 24
tf_ph = 60 * 48
opt_rk = False
tank = build_initial_ipopt_object(tank, tank_pars, dt_opt, k_baseline, k_MELT, k_IDLE, t0, tf_ph, max_melt, n_s, price_slope, regime_slope, opt_rk)


# Build initial switches
# Build initial values - Other methods could be considered
idle = tf_ph * np.linspace(0.1,0.9,n_s) # Spread out on the whole interval
melt = idle - max_melt/n_s * 0.866 # Assign melt period to a little before idle
switch0_dap = np.concatenate((melt,idle)) # put together


switch0 = switch0_dap
switch0

tank.set_p_optimize(switch0)



#%%
# SIMUALTE
#n_days = 2
n_skip = 20 #int(1/ och_sim)
start_date = '2018-01-01 12:00:00'
dt_stoch_sim = 0.1
seed = 1235

# Derive convenient parameters
np.random.seed(1234)
n_per_day =  int(1/n_skip * 1/dt_stoch_sim *60 * 24)
t0_sim = t0
t0_ph = t0
total_sim_time = tf_sim - t0_sim

# Initialize processes
nx_true = x0_true.size
nx_model = x0_model.size

# Initialzie history
history = {}
history['X_true'] = np.zeros([n_days, nx_true, n_per_day])
history['X_true_idle'] = np.zeros([n_days, nx_true, n_per_day])
history['X_true_init_switch'] = np.zeros([n_days, nx_true, n_per_day])
history['X_model_dap'] = np.zeros([n_days, nx_model, n_per_day])

history['Z_true'] = np.zeros([n_days, 1, n_per_day])
history['Z_true_init_switch'] = np.zeros([n_days, 1, n_per_day])
history['Z_true_idle'] = np.zeros([n_days, 1, n_per_day])
history['Z_model_dap'] = np.zeros([n_days, 1, n_per_day])

history['T'] = np.zeros([n_days, n_per_day])

history['SWITCHES_dap'] = np.zeros([n_days, 2 * n_s])
history['SWITCHES_init_switch'] = np.zeros([n_days, 2 * n_s])
history['SWITCHES_rk'] = np.zeros([n_days, 2 * n_s])
history['PRICES_dap'] = np.zeros([n_days, 48])

history['price_model_dap'] = np.zeros([n_days,n_per_day])
history['price_true'] = np.zeros([n_days,n_per_day])
history['price_true_init_switch'] = np.zeros([n_days,n_per_day])
history['price_true_idle'] = np.zeros([n_days,n_per_day])

history['price_model_dap_each_hour'] = np.zeros([n_days,n_per_day])
history['price_true_each_hour'] = np.zeros([n_days,n_per_day])
history['price_true_each_hour_init_switch'] = np.zeros([n_days,n_per_day])
history['price_true_each_hour_idle'] = np.zeros([n_days,n_per_day])

history['cons_model_dap'] = np.zeros([n_days,n_per_day])
history['cons_true'] = np.zeros([n_days,n_per_day])
history['cons_true_init_switch'] = np.zeros([n_days,n_per_day])
history['cons_true_idle'] = np.zeros([n_days,n_per_day])

history['cons_model_dap_each_hour'] = np.zeros([n_days,n_per_day])
history['cons_true_each_hour'] = np.zeros([n_days,n_per_day])
history['cons_true_each_hour_init_switch'] = np.zeros([n_days,n_per_day])
history['cons_true_each_hour_idle'] = np.zeros([n_days,n_per_day])

history['opt_time'] = np.zeros([n_days,1])
history['sim_time'] = np.zeros([n_days,1])

history['start_date'] = start_date

price_true_lasthour = 0.
price_true_lasthour_init_switch = 0.
price_true_lasthour_idle = 0.
price_promised_lasthour = 0.

cons_true_lasthour = 0.
cons_true_lasthour_init_switch = 0.
cons_true_lasthour_idle = 0.
cons_promised_lasthour = 0.

start_true = np.array(x0_true)
start_true_init_switch = np.array(x0_true)
start_true_idle = np.array(x0_true)
start_promised = np.array(x0_model)

idx_offset = np.where(prices.index == start_date)[0][0]


for day in range(n_days):
    print('  ----- Simulating day ' + str(day) + ' ----  ')
    logfile = open("../results/current_run_og_ANN_"+str(model_sys)+".txt","a")
    logfile.write("Simulating day " + str(day) + '\n')
    logfile.write("--------------\n")
    logfile.close()
    
    t0_sub = t0_sim/24.
    tf_sub = tf_sim/24.
    
    # Extract prices for optimization
    idx_offset = np.where(prices.index == start_date)[0][0]
    idx = np.arange(48) + day * 24 + idx_offset
    q_dap = np.array(prices['spot'][idx]).astype(np.float) # * 1/1000000
    
    # Insert new prices in optimization object
    p_dynamic = tank.get_p_dynamic()
    for k in range(0, 48):
        p_dynamic[k] = q_dap[k]
        #p_dynamic[48 + k] = q_rk[k]
        tank.set_p_dynamic(p_dynamic)
        
    # Initial state of the system
    start_promised = start_promised # Should be estimated with Kalman Filter
    tank.set_x0(np.append(start_promised,0)) 

    # Find optimal switches
    init_switch = tank.get_p_optimize()
    print(' ... Started optimizing')
    time_start = time.time()
    
    input_to_network_dap = scaler.transform(np.concatenate((start_promised,q_dap)).reshape(1,-1))

    switch_promised = predict_switch_ann_og(model,input_to_network_dap)

    
    #tank.solve()
    history['opt_time'][day] = time.time() - time_start
    print(' ... Finished optimizing')
    #opt_switch = tank.get_p_optimize_ipopt()
    #switch_promised = opt_switch[:2*n_s]
    #switch_used = opt_switch[2*n_s:]
    
    
    #switch_opt_dap = np.sort(np.random.uniform(0,tf_ph,2*n_s))
    #switch_promised = np.concatenate(derive_regimes(switch_opt_dap,0,0))
    #print(switch_opt_dap)

    #switch_opt_rk = np.sort(np.random.uniform(0,tf_ph,2*n_s))
    #switch_used = np.concatenate(derive_regimes(switch_opt_rk,0,0))
    #print(switch_opt_rk)
    
    
    # Simulate process and compute prices
    print(' ... Simulating processes')
    time_start = time.time()
    for hour in range(24):

        # ---------------------------
        # Simulate 1 day
        # ---------------------------
        # Make solution feasible
        switch_promised_filtered = makeANNFeasible(switch_promised)

        # Solve SDEs

        # True - Optimal switches
        if (switch_promised_filtered.size > 0):
            T_tmp, X_true, Y_true, Z_true = stochasticSimulation(system_true,switch_promised_filtered,start_true,t0_sub,tf_sub,dt_stoch_sim)
        else:
            T_tmp, X_true, Y_true, Z_true = stochasticSimulation_IDLE(system_true,start_true,t0_sub,tf_sub,dt_stoch_sim)

        # True - Always idle
        T_tmp_idle, X_true_idle, Y_true_idle, Z_true_idle = stochasticSimulation_IDLE(system_true,start_true_idle,t0_sub,tf_sub,dt_stoch_sim)

        # Expected  - Optimal switches
        T_tmp1, X_promised, Z_promised = sol_ivp_wrapper_discrete(system_model,start_promised,switch_promised_filtered,t0_sub,tf_sub,T_tmp)

        # True - Initial switches
        T_tmp_init_switch, X_true_init_switch, Y_true_init_switch, Z_true_init_switch = stochasticSimulation(system_true,init_switch,start_true_init_switch,t0_sub,tf_sub,dt_stoch_sim)


        # -----------------------------
        # Compute total consumption
        # -----------------------------
        # True
        cons_momental_true, cons_acc_true = consumption(Z_true[0],T_tmp,switch_promised,k_baseline,k_MELT,k_IDLE)

        # True - init switch
        cons_momental_true_init_switch, cons_acc_true_init_switch = consumption(Z_true_init_switch[0],T_tmp,init_switch,k_baseline,k_MELT,k_IDLE)
        
        # True IDLE - zero price of running the tank
        cons_momental_true_idle, cons_acc_true_idle = consumption(Z_true_idle[0],T_tmp,switch_promised,k_baseline,0,0)

        # Promised
        cons_momental_promised, cons_acc_promised = consumption(Z_promised[0],T_tmp,switch_promised,k_baseline,k_MELT,k_IDLE)

        # Compute payed prices
        price_payed = 1/1000000 *( cons_acc_true * q_dap[hour])
        price_payed_init_switch = 1/1000000 *( cons_acc_true_init_switch * q_dap[hour])
        price_payed_idle = 1/1000000 *( cons_acc_true_idle * q_dap[hour])
        price_promised = 1/1000000 *( cons_acc_promised * q_dap[hour])


        # -----------------------------
        # Save monitored variables
        # -----------------------------
        
        start = int(1/n_skip*1/dt_stoch_sim*hour*60)
        end = int(1/n_skip*1/dt_stoch_sim*(hour+1)*60)
        history['Z_true'][day][0][start:end] = Z_true[:,::n_skip]
        history['Z_true_init_switch'][day][0][start:end] = Z_true_init_switch[:,::n_skip]
        history['Z_true_idle'][day][0][start:end] = Z_true_idle[:,::n_skip]
        history['Z_model_dap'][day][0][start:end] = Z_promised[:,::n_skip]
        history['X_true'][day][:,start:end] = X_true[:,::n_skip]
        history['X_true_init_switch'][day][:,start:end] = X_true_init_switch[:,::n_skip]
        history['X_true_idle'][day][:,start:end] = X_true_idle[:,::n_skip]
        history['X_model_dap'][day][:,start:end] = X_promised[:,::n_skip]
        
        history['T'][day][start:end] = T_tmp[::n_skip] # Same in each day
        history['SWITCHES_dap'][day,:] = switch_promised
        history['SWITCHES_init_switch'][day,:] = init_switch
        history['PRICES_dap'][day,:] = q_dap

        # Price
        history['price_true'][day][start:end] = price_payed[::n_skip] + price_true_lasthour
        history['price_true_init_switch'][day][start:end] = price_payed_init_switch[::n_skip] + price_true_lasthour_init_switch
        history['price_true_idle'][day][start:end] = price_payed_idle[::n_skip] + price_true_lasthour_idle
        history['price_model_dap'][day][start:end] = price_promised[::n_skip] + price_promised_lasthour

        history['price_true_each_hour'][day][start:end] = price_payed[::n_skip]
        history['price_true_each_hour_init_switch'][day][start:end] = price_payed_init_switch[::n_skip]
        history['price_true_each_hour_idle'][day][start:end] = price_payed_idle[::n_skip]
        history['price_model_dap_each_hour'][day][start:end] = price_promised[::n_skip]

        # Consumption
        history['cons_true'][day][start:end] = cons_acc_true[::n_skip] + cons_true_lasthour
        history['cons_true_init_switch'][day][start:end] = cons_acc_true_init_switch[::n_skip] + cons_true_lasthour_init_switch
        history['cons_true_idle'][day][start:end] = cons_acc_true_idle[::n_skip] + cons_true_lasthour_idle
        history['cons_model_dap'][day][start:end] = cons_acc_promised[::n_skip] + cons_promised_lasthour

        history['cons_true_each_hour'][day][start:end] = cons_acc_true[::n_skip]
        history['cons_true_each_hour_init_switch'][day][start:end] = cons_acc_true_init_switch[::n_skip]
        history['cons_true_each_hour_idle'][day][start:end] = cons_acc_true_idle[::n_skip]
        history['cons_model_dap_each_hour'][day][start:end] = cons_acc_promised[::n_skip]
        

        # Update variables
        start_true = X_true[:,-1]
        start_true_init_switch = X_true_init_switch[:,-1]
        start_true_idle = X_true_idle[:,-1]
        start_promised = X_promised[:,-1] 
        t0_sub += total_sim_time/24.
        tf_sub += total_sim_time/24.

        price_true_lasthour = history['price_true'][day][end-1]
        price_true_lasthour_init_switch = history['price_true_init_switch'][day][end-1]
        price_true_lasthour_idle = history['price_true_idle'][day][end-1]
        price_promised_lasthour = history['price_model_dap'][day][end-1]

        cons_true_lasthour = history['cons_true'][day][end-1]
        cons_true_lasthour_init_switch = history['cons_true_init_switch'][day][end-1]
        cons_true_lasthour_idle = history['cons_true_idle'][day][end-1]
        cons_promised_lasthour = history['cons_model_dap'][day][end-1]
        

    history['sim_time'][day] = time.time() - time_start

    history['end_date'] = prices.index[idx[-1]]

    filenname = '../results/sim_history/og_ann_history_l2(' + start_date + ')_(' + str(n_days) + '_days)' + '_(price_slope_' + str(price_slope) +  ')_(regime_slope_' + str(regime_slope) +  ')_(seed_' + str(seed) +  ')_(n_s_' + str(n_s) + ')_(sys_model_' +  sys_mod + ')_(sys_true_' + sys_true + ')' +  '.npy'
    np.save(filenname,history)


1
3
  ----- Simulating day 0 ----  
 ... Started optimizing
 ... Finished optimizing
 ... Simulating processes
  ----- Simulating day 1 ----  
 ... Started optimizing
 ... Finished optimizing
 ... Simulating processes
  ----- Simulating day 2 ----  
 ... Started optimizing
 ... Finished optimizing
 ... Simulating processes
  ----- Simulating day 3 ----  
 ... Started optimizing
 ... Finished optimizing
 ... Simulating processes
  ----- Simulating day 4 ----  
 ... Started optimizing
 ... Finished optimizing
 ... Simulating processes
  ----- Simulating day 5 ----  
 ... Started optimizing
 ... Finished optimizing
 ... Simulating processes


ValueError: operands could not be broadcast together with shapes (724,) (600,) 